In [1]:
import pandas as pd
import numpy as np
from matrixprofile import *
from matrixprofile.discords import discords
from matplotlib import pyplot as plt
from scipy.io import arff
from binarytree import Node
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy
from math import log, e
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
#VERSIONE CON USO DEI MOTIF,DISCORD O ENTRAMBI E CON MODULARITA' NELLA RIMOZIONE DEI CANDIDATI GIA SCELTI
# AGGIUNTA ANCHE LA FUNZIONE MIA PER SCEGLIERE IL BEST ATTRIBUTE E RELATIVO VALORE

In [3]:
def retrieve_all(Ts,window_size): #fornita la Ts calcola e restituisce mp, motifs, motifs_distances e discords
    Ts=Ts[0:127] #rimuovo l'attributo "classe"

    dfMP = pd.DataFrame(Ts).astype(float) # genero Dframe per lavorarci su, DA CAPIRE PERCHE SERVE FLOAT
    mp, mpi = matrixProfile.stomp(dfMP[0].values,window_size) #OK STOMP

    #PREPARO TUPLA DA PASSARE ALLA FUN MOTIF (RICHIEDE TUPLA FATTA DA MP E MPI)
    tupla=mp,mpi

    mot, motif_dist  =motifs.motifs(dfMP[0].values,tupla,2)

    #CALCOLO MOTIFS
   # print('Motifs starting position: '+str(mot)+ ' Motifs values (min distances): '+str(motif_dist))
   # print(" ")

    #CALCOLO DISCORDS
    dis= discords(mp,window_size,2)
    #print('Discords starting position: '+str(dis))
    
    tupla=mp,mot,motif_dist,dis
    return tupla

In [4]:
#ogni motif e identificato da almeno due indici di partenza nella Ts, ne prendo uno solo rappresentativo
#genero poi struttura contenente gli indici di partenza di tutti i candidati

def candidateFilter(CandidatesList): 
    counterNumberMotif=0
    counterNumberDiscord=0
    l2=np.array([])
    for i in range (len(CandidatesList['Motif'])): #per ogni entry (per ogni record)
        numMotif=len(CandidatesList['Motif'].iloc[i])
        numDiscord=len(CandidatesList['Discord'].iloc[i])
        counterNumberDiscord+=numDiscord
        #print(numMotif)
        for j in range (numMotif): # per ogni lista di motif
            l1=CandidatesList['Motif'].iloc[i] #prima lista
            l2=np.append(l2,l1[j][0]) #prendo primo valore di ogni lista
            counterNumberMotif+=1
            
    
        CandidatesList['Motif'].iloc[i]=l2
        l2=np.array([]) #svuoto array
    
    return CandidatesList,counterNumberMotif,counterNumberDiscord

In [5]:
#
def buildCandidatesUsedList(CandidatesList,numberOfMotif,numberOfDiscord):
    CandidatesUsedList=pd.DataFrame(columns=['Used'],index=range(0,numberOfMotif+numberOfDiscord))
    boolList=[False] * (numberOfMotif+numberOfDiscord)
    CandidatesUsedList['Used']= boolList
    return CandidatesUsedList
                                    

In [6]:
def getDataStructures(df,window_size):
    #trasformo da stringa a numero il campo target 
    le = LabelEncoder()
    num_classes = le.fit_transform(df['target'])
    df['target']=num_classes
    df['TsIndex']=np.arange(len(df))
    window_size=5
    #diz={'Motif':[],'Motif-Dist':[],'Discord':[]}
    diz={'Motif':[],'Discord':[]}

    #CALCOLO MOTIF E DISCORD E LI INSERISCO NEL DIZIONARIO
    for i in range(len(df)):
        Ts = np.array(df.iloc[i][0:-2].values)
        mp,mot,motif_dist,dis = retrieve_all(Ts,window_size)
        diz['Motif'].insert(i, mot)
        diz['Discord'].insert(i, dis)

    #GENERO DFRAME DA DIZIONARIO

    CandidatesList = pd.DataFrame(diz)
    CandidatesList,numberOfMotif,numberOfDiscord=candidateFilter(CandidatesList)
    CandidatesUsedList=buildCandidatesUsedList(CandidatesList,numberOfMotif,numberOfDiscord)

    print('Candidati estratti')
    #print(MotifsList['Motif'])
    print(CandidatesList)
    print(numberOfMotif,numberOfDiscord)
    print(CandidatesUsedList)
    
    return mp,CandidatesList,numberOfMotif,numberOfDiscord,CandidatesUsedList

In [7]:
#per ogni Ts calcolo Dprofile con ogni candidato e inserisco la distanza minima con candidato i-esimo nella colonna i-esima
def computeSubSeqDistance(dataset,CandidatesList):
    
    #quantifico il num di candidati e in base a tale valore genero colonne per dfForDTree
    numberOfCandidates=0
    for i in range(len(CandidatesList)):
            numberOfCandidates+=len(CandidatesList['Motif'].loc[i])
            numberOfCandidates+=len(CandidatesList['Discord'].loc[i])
    columnsList=np.arange(numberOfCandidates)
    columnsList2=list()
    lastAttribute=['TsIndex','class']
    prefix='cand'
    for i in columnsList:
        columnsList2.append(prefix+str(i)) 
    columnsList2.append('TsIndex')
    columnsList2.append('class')
    dfForDTree=pd.DataFrame(columns=columnsList2,index=range(0,len(dataset)))

    #per ogni Ts, scandisco ogni candidato e calcolo la distanza minore 
    for i in range(len(dataset)):
        #acquisisco la Ts
        TsToCompare = np.array(dataset.iloc[i].values) 
        classValue=TsToCompare[128]
        TsToCompare=TsToCompare[0:128]
        dfForDTree['TsIndex'].iloc[i]=i
        dfForDTree['class'].iloc[i]=classValue
        counter=0
        #scandisco e calcolo distanza dai motif
        for j in range(len(CandidatesList)):
            numMotif=len(CandidatesList['Motif'].iloc[j])
            numDiscord=len(CandidatesList['Discord'].iloc[j])
            for k in range(numMotif):
                l1=CandidatesList['Motif'].iloc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                counter+=1
            for k in range(numDiscord):
                l1=CandidatesList['Discord'].iloc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                counter+=1
        
   # print(counter)    
    return dfForDTree #columnsList2 restituito per generare poi dFrame in "Split" (struttura dframe)
                
                

In [8]:
#dataset (dframe): nella riga i: indice della ts di appartenenza, distanza tra candidato e Ts, e classe di appartenenza di Ts
#calcola entropia di un dataset basandosi sul num di classi esistenti
def computeEntropy(dataset):
    value,counts = np.unique(dataset['class'], return_counts=True)
    actualEntropy=entropy(counts, base=2)
    return actualEntropy

In [9]:
#calcola il gain tra entropia nodo padre e sommatoria entropia nodi figli (GAIN CALCOLATO SUL VALORE DELL'ATTRIBUTO)
def computeGain(entropyParent,LenDatasetParent,Dleft,Dright):
    entropyLeft=computeEntropy(Dleft)
    entropyRight=computeEntropy(Dright)
    gain=entropyParent
    summation=( ((len(Dleft)/LenDatasetParent)*entropyLeft) +  ((len(Dright)/LenDatasetParent)*entropyRight) )
    #print('entropyParent: '+str(entropyParent))
    #print('SUMMATION: '+str(summation))
    gain=gain-summation
    return gain

In [10]:
#SPLIT SLAVE
#effettua lo split del dataset sul attributo e valore fornito
def split(dataset,attribute,value): 
    columnsList=dataset.columns.values
    dizLeft=pd.DataFrame(columns=columnsList)
    dizRight=pd.DataFrame(columns=columnsList)
    for i in range(len(dataset)):
        if dataset.iloc[i][attribute] < value:
            dizLeft = dizLeft.append(dataset.iloc[i], ignore_index=True)
        else:
            dizRight = dizRight.append(dataset.iloc[i], ignore_index=True)
    return dizLeft, dizRight

In [117]:
# riceve dframe con mutual_information(gain) e in base al candidatesGroup scelto, determina il miglior attributo su cui splittare
# che non è stato ancora utilizzato
def getBestIndexAttribute(vecMutualInfo,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord):
        
    #ordino i candidati in base a gain decrescente
    
    vecMutualInfo=vecMutualInfo.sort_values(by='gain',ascending = False)
    
    #scandisco i candidati fino a trovare il candidato con miglior gain che non è ancora stato usato
    
    bestIndexAttribute=-1
    i=0
    
    #cicla fin quando trova candidato libero con gain maggiore
    while(bestIndexAttribute==-1 and i<len(vecMutualInfo)):    
        attributeToVerify=int(vecMutualInfo.iloc[i]['attribute'])
        print('attributo esaminato: ')
        print(attributeToVerify)
        if(CandidatesUsedListTrain.iloc[attributeToVerify]['Used']==False):
            bestIndexAttribute=attributeToVerify
            splitValue=vecMutualInfo.iloc[i]['splitValue']
            CandidatesUsedListTrain.iloc[attributeToVerify]=True #settando a true il candidato scelto, non sarà usato in seguito
        else:
            i+=1
    
    return bestIndexAttribute,splitValue
            
    
        
        

In [118]:
def computeMutualInfo(datasetForMutual,candidatesGroup,numberOfMotif,numberOfDiscord):
    
    #definisco lista di indici inserire nella colonna 'attribute'
    if(candidatesGroup==0):
        candidatesIndex=range(numberOfMotif)
        numAttributes=numberOfMotif
    elif(candidatesGroup==1):
        candidatesIndex=range(numberOfMotif,numberOfMotif+numberOfDiscord)
        numAttributes=numberOfDiscord
    else:
        candidatesIndex=range(numberOfMotif+numberOfDiscord)
        numAttributes=numberOfMotif+numberOfDiscord
    
    columns=datasetForMutual.columns
    dframe=pd.DataFrame(columns=['attribute','splitValue','gain'],index=range(len(columns)-1))
    entropyParent=computeEntropy(datasetForMutual)

    
    
    y=datasetForMutual['class']
    
    #per ogni attributo, ordino il dframe sul suo valore
    #scandisco poi la y e appena cambia il valore di class effettuo uno split, memorizzando il best gain
    
    for i in range(len(columns)-1):
        bestGain=-1
        bestvalueForSplit=0
        previousClass=-1 #deve essere settato ad un valore non presente nei class value
        attribute=columns[i]
        datasetForMutual=datasetForMutual.sort_values(by=attribute,ascending = True)    
        
       
        
        for j in range(len(y)):
            if(j==0):
                previousClass=y[j]
                continue
            else:
                if(y[j]!=previousClass):
                    testValue=datasetForMutual.iloc[j][attribute]
                    Dleft,Dright=split(datasetForMutual,attribute,testValue)
                    actualGain=computeGain(entropyParent,len(datasetForMutual),Dleft,Dright)
                    if(actualGain > bestGain):
                        bestGain=actualGain
                        bestvalueForSplit=testValue
               
                previousClass=y[j] 
        # memorizzo in posizione i-esima lo split migliore e relativo gain
        
        dframe.iloc[i]['splitValue']=bestvalueForSplit
        dframe.iloc[i]['gain']=bestGain
        
    
    
    dframe['attribute']=candidatesIndex
    return dframe
        
        
    
    
    

In [119]:
#SPLIT INTERMEDIO
#dato il dataset, cerca il miglior attributo e relativo valore (optimal split point) su cui splittare
# restituiendo il dataset splittato e i valori trovati
def findBestAttributeValue(dataset,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate):
    #cerca e restituisce attributo migliore su cui splittaree relativo valore ottimale (optimal split point)
    #CANDIDATE GROUP permette di scegliere se usare come candidati 0=motifs 1=discord 2=entrambi
    bestGain=0
    actualGain=0
    bestvalueForSplit=0
    y = dataset['class'].values
    y=y.astype('int')
    entropyParent=computeEntropy(dataset)
    
    #trovo best Attribute
    numAttributes=len(dataset.columns.values)
    numAttributes-=2 #tolgo i due attributi TsIndex e class dal Dframe
    datasetForMutual=pd.DataFrame()
    
    #preparo il Dframe da passare a mutual_info_classif, settando se scegliere tra motifs/discord/entrambi
    
    if(candidatesGroup==0): #solo motifs
        dataset=dataset.iloc[:,np.r_[:numberOfMotif]]
    elif(candidatesGroup==1):
        datasetForMutual=dataset.iloc[:,np.r_[numberOfMotif:numberOfMotif+numberOfDiscord]]
    else:
        datasetForMutual=dataset.iloc[:,np.r_[:numAttributes]]

    #calcolo gain e miglior valore di split per ogni attributo
        
    datasetForMutual['class']=y
    
    vecMutualInfo=computeMutualInfo(datasetForMutual,candidatesGroup,numberOfMotif,numberOfDiscord)
    
    #se rimuovo candidati, faccio scegliere migliore non ancora utilizzato
    
    if(removeUsedCandidate==1): 
        indexBestAttribute,bestValueForSplit=getBestIndexAttribute(vecMutualInfo,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord)
    else: #se non rimuovo candidati, mi basta prendere il primo
        vecMutualInfo=vecMutualInfo.sort_values(by='gain',ascending = False)
        indexBestAttribute=vecMutualInfo.iloc[0]['attribute']
        bestValueForSplit=vecMutualInfo.iloc[0]['splitValue']
    
    
    splitValue=bestValueForSplit
    Dleft,Dright=split(dataset,indexBestAttribute,bestValueForSplit)
    
    
    return [indexBestAttribute,splitValue,Dleft,Dright]
    

In [120]:
#SPLIT MASTER
# funzione ricorsiva che implementa la creazione dell'albero di classificazione
# memorizza in ogni nodo: attributo, valore attributo su cui splitto, entropia nodo, num pattern
# memorizza in ogni foglia: entropia nodo, num pattern, classe nodo

# VERSIONE CHE RIMUOVE I CANDIDATI QUANDO VENGONO SCELTI 

def buildTree(actualNode,dataset,maxDepth, minSamplesLeaf, depth,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate):
    #caso base: num pattern < soglia minima || profondità massima raggiunta => genero foglia con media delle classi
    #DATASET HA SEMPRE ALMENO UN PATTERN
    boolValue=checkIfIsLeaf(dataset)
    if(len(dataset)<minSamplesLeaf or depth>=maxDepth or boolValue==True ):
        average = sum(dataset['class'].values) / len(dataset['class'].values)
        classValue = round(average)
        numPattern=len(dataset)
        entropy=computeEntropy(dataset)
        
        nodeInfo=list()
        nodeInfo.append(classValue)
        nodeInfo.append(numPattern)
        nodeInfo.append(entropy)
    
        actualNode.data=nodeInfo
        actualNode.value=-1
        actualNode.left=None
        actualNode.right=None
        #print(dataset['class'])
        return 
    #caso ricorsivo in cui si può splittare
    else:
        
        returnList=findBestAttributeValue(dataset,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
        print('DENTRO BUILD FIND HA RESTITUITO: ')
        print(returnList)
        indexChosenAttribute=returnList[0]
        attributeValue=returnList[1]
        Dleft=returnList[2]
        Dright=returnList[3]
        numPattern=len(dataset)
        entropy=computeEntropy(dataset)
        attributeList.append(indexChosenAttribute)
        
        #memorizzo nel nodo l'attributo, il valore e altre info ottenute dallo split
        
        nodeInfo=list()
        nodeInfo.append(attributeValue)
        nodeInfo.append(numPattern)
        nodeInfo.append(entropy)
        actualNode.data=nodeInfo
        actualNode.value=(indexChosenAttribute)
        
        #se possibile richiamo ricorsivamente sul nodo dx e sx figlio
        if(len(Dleft)>0):
            actualNode.left=Node(indexChosenAttribute)
            buildTree(actualNode.left,Dleft,maxDepth, minSamplesLeaf, depth+1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
        
        if(len(Dright)>0):
            actualNode.right=Node(indexChosenAttribute)
            buildTree(actualNode.right,Dright,maxDepth, minSamplesLeaf, depth+1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
            
        

In [121]:
#verifica se dataset, ha pattern appartenenti ad una sola classe => è gia foglia
def checkIfIsLeaf(dataset):
    isLeaf=True
    entropy=computeEntropy(dataset)
    if(entropy>0):
        isLeaf=False
    return isLeaf

In [122]:
#effettua il primo passo dell'algo di generazione dell'albero, richiama ricorsivamente sui figli
# VERSIONE CHE NON RIMUOVE I CANDIDATI QUANDO VENGONO SCELTI 
def startAlgo(dfForDTree,candidatesGroup,CandidatesUsedListTrain,maxDepth,minSamplesLeaf,numberOfMotif,numberOfDiscord,removeUsedCandidate):
    
    #inizio algo per nodo radice
    returnList=findBestAttributeValue(dfForDTree,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
    indexChosenAttribute=returnList[0]
    attributeValue=returnList[1]
    Dleft=returnList[2]
    Dright=returnList[3]
    print('DENTRO START FIND HA RESTITUITO: ')
    print(returnList)
    attributeList.append(indexChosenAttribute)
    root=Node(indexChosenAttribute)
    numPattern=len(dfForDTree)
    entropy=computeEntropy(dfForDTree)
        
    #memorizzo nel nodo l'attributo, il valore e altre info ottenute dallo split
        
    nodeInfo=list()
    nodeInfo.append(attributeValue)
    nodeInfo.append(numPattern)
    nodeInfo.append(entropy)
    root.data=nodeInfo
    
    root.left=Node(indexChosenAttribute)
    root.right=Node(indexChosenAttribute)
    
    #chiamata ricorsiva
    if(len(Dleft)>0):
        buildTree(root.left,Dleft,maxDepth, minSamplesLeaf,1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
    if(len(Dright)>0):
        buildTree(root.right,Dright,maxDepth, minSamplesLeaf, 1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
    return root

In [123]:
#stampa dell'albero
def printAll(Root):
    if(Root.left==None and Root.right==None):
        print('foglia')
    print('Nodo: '+str(Root.value))
    df=Root.data
    print(df)
    print("\n")
    if(Root.left!=None):
        printAll(Root.left)
    if(Root.right!=None):
        printAll(Root.right)

In [124]:
def predict(testDataset,root):
    print('START PRED')
    #preparo dataset 
    numAttributes=len(testDataset.columns.values)
    numAttributes-=2 #per prendere solo gli attributi utili a xTest
    yTest=testDataset.iloc[:]['class'].values
    yPredicted=np.zeros( len(yTest) )
    xTest=testDataset.iloc[:,np.r_[:numAttributes]]
    
    #effettuo predizione per ogni pattern
    
    for i in range(len(xTest)):
        pattern=xTest.iloc[i]
        yPredicted[i]= treeExplorer(pattern,root)
    
    yTest = yTest.astype(int)
    yPredicted = yPredicted.astype(int)
    
    return yTest,yPredicted
        

In [125]:
def treeExplorer(pattern,node):
    #caso base, node è foglia
    if(node.value==-1):
        return int(node.data[0])
    else:
    #caso ricorsivo
        attr='cand'+str(node.value)
        if(pattern[attr] < node.data[0]):
            return treeExplorer(pattern,node.left)
        else:
            return treeExplorer(pattern,node.right)

In [126]:
def computeSubSeqDistanceForTest(datasetTest,datasetTrain,attributeList,CandidatesList,numberOfMotif,numberOfDiscord):
    #quantifico il num di candidati usati dall'albero e in base a tale valore genero colonne per dfForDTree
      #quantifico il num di candidati e in base a tale valore genero colonne per dfForDTree
    columnsList2=list()
    prefix='cand'
    for i in attributeList:
        columnsList2.append(prefix+str(i)) 
    columnsList2.append('TsIndex')
    columnsList2.append('class')
    dfForDTree=pd.DataFrame(columns=columnsList2,index=range(0,len(datasetTest)))

    #per ogni Ts, scandisco ogni candidato e calcolo la distanza minore 
    for i in range(len(datasetTest)):
        #acquisisco la Ts
        TsToCompare = np.array(datasetTest.iloc[i].values) 
        classValue=TsToCompare[128]
        TsToCompare=TsToCompare[0:128]
        dfForDTree['TsIndex'].iloc[i]=i
        dfForDTree['class'].iloc[i]=classValue
        counter=0
        #scandisco e calcolo distanza dai motif
        for z in range(len(attributeList)):
            candidateIndex=attributeList[z]
            counter=0
            for j in range(len(CandidatesList)):
                numMotif=len(CandidatesList['Motif'].iloc[j])
                numDiscord=len(CandidatesList['Discord'].iloc[j])
                for k in range(numMotif):
                    if(counter==candidateIndex):
                        l1=CandidatesList['Motif'].iloc[j] #lista di indice i in motifDiscordList
                        startingIndex=l1[k] #indice di inizio del motif
                        TsContainingCandidateShapelet = np.array(datasetTrain.iloc[j].values) #Ts contenente candidato shapelet
                        Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                        minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                        dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                    counter+=1
                        
                for k in range(numDiscord):
                    if(counter==candidateIndex):
                        l1=CandidatesList['Discord'].iloc[j] #lista di indice i in motifDiscordList
                        startingIndex=l1[k] #indice di inizio del motif
                        TsContainingCandidateShapelet = np.array(datasetTrain.iloc[j].values) #Ts contenente candidato shapelet
                        Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                        minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                        dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                    counter+=1
                    



    le = LabelEncoder()        
    num_classes = le.fit_transform(dfForDTree['class'])
    dfForDTree['class']=num_classes
            
    return dfForDTree #columnsList2 restituito per generare poi dFrame in "Split" (struttura dframe)
                

    

In [127]:
#ACQUISISCO STRUTTURE DATI DEL TRAINING SET
dataset = arff.loadarff('CBF/CBF_TRAIN.arff')
dfTrain=pd.DataFrame(dataset[0])
window_size=5
mpTrain,CandidatesListTrain,numberOfMotifTrain,numberOfDiscordTrain,CandidatesUsedListTrain=getDataStructures(dfTrain,window_size)
dfForDTree=computeSubSeqDistance(dfTrain,CandidatesListTrain)


Candidati estratti
            Motif    Discord
0     [9.0, 26.0]   [61, 27]
1   [105.0, 54.0]   [8, 119]
2    [17.0, 40.0]    [31, 1]
3    [48.0, 12.0]   [23, 86]
4          [27.0]   [38, 62]
5    [45.0, 13.0]  [119, 26]
6     [7.0, 37.0]   [98, 76]
7     [2.0, 24.0]    [36, 8]
8     [1.0, 29.0]   [23, 32]
9     [50.0, 0.0]   [38, 59]
10   [11.0, 17.0]   [80, 23]
11    [31.0, 7.0]  [114, 54]
12         [53.0]   [61, 10]
13    [79.0, 6.0]  [105, 38]
14         [59.0]    [98, 8]
15    [2.0, 45.0]     [1, 8]
16    [28.0, 3.0]    [5, 88]
17   [14.0, 35.0]  [118, 85]
18    [13.0, 3.0]   [111, 8]
19    [19.0, 6.0]   [92, 35]
20   [19.0, 39.0]   [26, 70]
21         [26.0]   [13, 35]
22   [18.0, 22.0]   [59, 25]
23   [27.0, 42.0]  [32, 104]
24    [8.0, 28.0]    [4, 95]
25         [16.0]   [87, 20]
26   [67.0, 59.0]   [33, 83]
27   [16.0, 39.0]   [75, 99]
28   [11.0, 41.0]   [47, 39]
29    [6.0, 15.0]  [117, 51]
55 60
      Used
0    False
1    False
2    False
3    False
4    False
..     ...

In [128]:
#COSTRUISCO DECISION TREE
candidatesGroup=1
albero=None
maxDepth=3
minSamplesLeaf=5
attributeList=list()
removeUsedCandidate=0
albero=startAlgo(dfForDTree,candidatesGroup,CandidatesUsedListTrain,maxDepth,minSamplesLeaf,numberOfMotifTrain,numberOfDiscordTrain,removeUsedCandidate)
print(albero)
print(attributeList)
printAll(albero)



<ipython-input-119-5526614809ae>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetForMutual['class']=y


DENTRO START FIND HA RESTITUITO: 
[110, 0.673469385586017,        cand0     cand1     cand2     cand3     cand4     cand5     cand6  \
0   0.512876  0.364486  0.614296  1.026370  0.764749  0.999454  0.888062   
1   0.476238  0.450861  0.357785  0.487631  0.842248  0.532013  0.824725   
2   0.640046  0.922417  0.266988  0.632237  0.898213  0.783500  0.923515   
3   0.852676  0.447612  0.809304  0.724721  1.032211  0.752910  0.578409   
4   0.820811  0.221007  0.385794  0.907680  0.511086  0.422400  0.324884   
5   0.531972  0.470080  0.642830  0.896841  0.908945  0.593944  0.976889   
6   0.528394  0.574339  0.706176  0.788202  0.456593  0.258866  0.901551   
7   0.392114  0.471061  0.355923  0.366186  0.660584  0.669648  0.798032   
8   0.874073  0.345060  1.031679  0.819838  0.555663  0.722166  0.500529   
9   0.384127  0.511508  0.951830  0.746768  1.085555  0.522401  0.661212   
10  0.653865  0.636918  0.930848  0.399888  0.798290  0.600730  0.547853   
11  0.410703  0.519167  0.834

<ipython-input-119-5526614809ae>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetForMutual['class']=y


DENTRO BUILD FIND HA RESTITUITO: 
[114, 0.665356316581676,       cand0     cand1     cand2     cand3     cand4     cand5     cand6  \
0  0.640046  0.922417  0.266988  0.632237  0.898213  0.783500  0.923515   
1  0.820811  0.221007  0.385794  0.907680  0.511086  0.422400  0.324884   
2  0.531972  0.470080  0.642830  0.896841  0.908945  0.593944  0.976889   
3  0.392114  0.471061  0.355923  0.366186  0.660584  0.669648  0.798032   
4  0.410703  0.519167  0.834683  0.566085  0.218233  0.741967  0.313779   
5  0.670882  0.451390  0.805692  0.655036  0.643457  1.135482  1.141732   

      cand7     cand8     cand9  ...   cand107   cand108   cand109   cand110  \
0  0.534674  0.545432  0.687134  ...  0.637623  0.727362  0.526756  0.537895   
1  0.721129  1.115667  0.517207  ...  0.263205  1.189301  0.394050  0.379042   
2  0.876546  0.796344  0.575945  ...  0.687575  0.989240  0.475861  0.448262   
3  0.658401  0.801001  0.182415  ...  0.681402  0.651866  0.811048  0.593873   
4  0.824740  0.

<ipython-input-119-5526614809ae>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetForMutual['class']=y


...

DENTRO BUILD FIND HA RESTITUITO: 
[106, 0.7926101688282069,       cand0     cand1     cand2     cand3     cand4     cand5     cand6  \
0  0.476238  0.450861  0.357785  0.487631  0.842248  0.532013  0.824725   
1  0.852676  0.447612  0.809304  0.724721  1.032211  0.752910  0.578409   
2  0.874073  0.345060  1.031679  0.819838  0.555663  0.722166  0.500529   
3  0.967778  0.562344  1.080780  0.390414  0.716793  0.454008  0.582436   
4  0.648161  0.727367  0.711043  0.271805  0.919921  0.957876  0.877619   

      cand7     cand8     cand9  ...   cand107   cand108       cand109  \
0  0.844056  0.519703  0.793168  ...  0.756261  0.578003  4.636798e-01   
1  0.772530  0.689880  0.869449  ...  0.921972  0.858750  1.016916e+00   
2  0.751791  0.722520  0.762199  ...  0.557949  0.952972  3.820108e-01   
3  0.356013  0.604935  0.525245  ...  0.804468  0.523680  4.843415e-01   
4  0.710307  0.577849  0.658635  ...  0.000000  0.000000  2.686345e-07   

    cand110   cand111   cand112   cand113   

<ipython-input-119-5526614809ae>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetForMutual['class']=y


DENTRO BUILD FIND HA RESTITUITO: 
[76, 0.36496414311772096,       cand0     cand1     cand2     cand3     cand4     cand5     cand6  \
0  0.661830  0.474448  0.544242  0.899456  0.882292  0.978614  0.447686   
1  0.385411  0.858093  0.949343  0.716721  0.276801  0.545861  0.384134   

      cand7     cand8     cand9  ...   cand107   cand108   cand109   cand110  \
0  0.816464  0.765592  0.752743  ...  0.703440  0.835687  0.986942  0.881203   
1  0.895927  0.711170  0.729926  ...  0.655771  0.837167  0.529089  0.835562   

    cand111   cand112   cand113   cand114  TsIndex  class  
0  0.617913  0.712832  0.438059  0.945463       19    2.0  
1  0.273202  0.663976  0.646987  1.051911       27    2.0  

[2 rows x 117 columns],        cand0     cand1     cand2     cand3         cand4         cand5  \
0   0.000000  0.000000  0.000000  0.000000  9.102327e-01  4.907781e-01   
1   0.983997  0.738271  0.658725  0.731558  1.526915e-07  3.956532e-07   
2   0.612164  0.630685  1.009647  0.683533  1.

<ipython-input-119-5526614809ae>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetForMutual['class']=y


DENTRO BUILD FIND HA RESTITUITO: 
[103, 0.34869675614135354,       cand0     cand1     cand2     cand3         cand4         cand5  \
0  0.000000  0.000000  0.000000  0.000000  9.102327e-01  4.907781e-01   
1  0.983997  0.738271  0.658725  0.731558  1.526915e-07  3.956532e-07   

     cand6     cand7     cand8     cand9  ...   cand107   cand108   cand109  \
0  0.68418  0.927677  0.784015  1.047104  ...  0.548972  0.819577  0.695128   
1  0.00000  0.000000  1.027420  0.654510  ...  0.474360  0.901841  0.767816   

    cand110   cand111   cand112   cand113   cand114  TsIndex  class  
0  1.158248  0.517805  0.718874  0.928936  0.602325        0    0.0  
1  1.060059  0.819484  0.787542  1.095482  0.658191        1    0.0  

[2 rows x 117 columns],       cand0     cand1     cand2     cand3     cand4     cand5     cand6  \
0  0.612164  0.630685  1.009647  0.683533  0.186662  0.394852  0.451492   
1  0.522029  0.789137  0.656120  0.472583  0.721269  0.872467  0.974722   
2  0.688860  0.635310

In [129]:
#GENERO STRUTTURE DATI PER TEST SET
dataset2 = arff.loadarff('CBF/CBF_TEST.arff')
window_size=5
dfTest = pd.DataFrame(dataset2[0]) #30 record su matrice da 128 attributi + 'b': classe appartenenza  
dfTest=dfTest.iloc[50:100] #ne prendo 50 altrimenti impiega tempo troppo lungo, sono 900 record totali

attributeList=sorted(attributeList)
dfForDTreeTest=computeSubSeqDistanceForTest(dfTest,dfTrain,attributeList,CandidatesListTrain,numberOfMotifTrain,numberOfDiscordTrain) 
print(dfForDTreeTest)
#CONTROLLA, DOVRRBBE ESSERE GIUSTO COSI, SE USO COMPUTE NORMALE CON dfTest SBAGLIO DATASET DI RIFERIMENTO 


      cand76   cand103   cand106   cand107   cand110   cand114 TsIndex  class
0    0.62867  0.547082  0.934706  0.518015  0.342897  0.545575       0      2
1    1.18544  0.658432   0.70932  0.509231  0.725863  0.844776       1      0
2   0.666738  0.488897  0.680409  0.928681  0.300672  0.532179       2      2
3   0.776877  0.226223   0.84024  0.842408   0.49576  0.475469       3      0
4   0.790711  0.846641  0.252335  0.562411  0.690176   0.88914       4      1
5   0.236352  0.832025  0.155866  0.671525  0.678225  0.978216       5      0
6   0.520489  0.896888    1.0778  0.682148  0.561388  0.889395       6      2
7   0.771316  0.832347  0.834699  0.654508  0.540481  0.800468       7      2
8   0.455719  0.871422  0.431466  0.597434  0.787613  0.425991       8      2
9     1.0889  0.583803  0.974125  0.386135  0.457872  0.897348       9      2
10     1.205  0.568212  0.928791  0.931941   0.85078  0.347987      10      1
11  0.300089  0.608506  0.455334  0.690218  0.425782  0.364568  

In [130]:
#EFFETTUO PREDIZIONE E MISURO RISULTATO

yTest,yPredicted=predict(dfForDTreeTest,albero)



for a,b in zip(yTest,yPredicted):
    print(a,b)
    
print(type(yPredicted))
print(type(yTest))
    
print(classification_report(yTest, yPredicted))
print('Accuracy %s' % accuracy_score(yTest, yPredicted))
print('F1-score %s' % f1_score(yTest, yPredicted, average=None))
confusion_matrix(yTest, yPredicted)    

START PRED
2 0
0 1
2 1
0 1
1 1
0 2
2 0
2 0
2 1
2 0
1 1
0 1
2 2
0 2
2 2
0 2
0 2
2 1
1 2
0 1
1 1
2 1
2 2
0 1
1 1
1 1
1 0
1 2
2 2
1 2
0 2
0 1
0 0
0 1
2 0
2 1
2 1
0 1
2 0
0 0
2 2
0 1
1 1
1 1
0 1
1 0
2 2
2 1
0 1
2 0
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
              precision    recall  f1-score   support

           0       0.18      0.11      0.14        18
           1       0.28      0.58      0.38        12
           2       0.43      0.30      0.35        20

    accuracy                           0.30        50
   macro avg       0.30      0.33      0.29        50
weighted avg       0.30      0.30      0.28        50

Accuracy 0.3
F1-score [0.13793103 0.37837838 0.35294118]


array([[ 2, 11,  5],
       [ 2,  7,  3],
       [ 7,  7,  6]], dtype=int64)